In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
print(selenium.__version__)

4.20.0


In [7]:
# Read data from excel
# Read the Excel file
file_path = '../cities_scrap.xlsx'  # Replace with your file path
sheet_name = 'Test'  # Replace with your sheet name
column_name_city = 'city'  # Replace with your column name
column_name_country = 'country'  # Replace with your column name

# Load the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the column to a list
city_list = df[column_name_city].tolist()
country_list = df[column_name_country].tolist()

# final list
city_names = []
for idx,city in enumerate(city_list):
    city_names.append(city_list[idx])
    city_names.append(str(city_list[idx])+'-'+str(country_list[idx]))

# city_names

In [8]:
# Initialize WebDriver

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up the WebDriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()  # Assuming you're using Chrome

# Base URL
url_base = 'https://www.numbeo.com/quality-of-life/in/'

# Initialize an empty DataFrame
df_capital_all = pd.DataFrame(columns=[
    'index',
    'city',
    'climate_index',
    'cost_of_living_index',
    'health_care_index',
    'pollution_index',
    'property_price_to_income_ratio',
    'purchasing_power_index',
    'safety_index',
    'traffic_commute_time_index',
    'quality_of_life_index'
])

In [9]:
# Fetch data using Selenium

for idx,city in enumerate(city_names):

    try:

        url_city = url_base + city_names[idx]
        driver.get(url_city)

        # Find Table element
        tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

        
        # Parse Table element into pandas DataFrame
        tbl_quality_html = tbl_quality.get_attribute('outerHTML')
        soup = BeautifulSoup(tbl_quality_html, 'html.parser')

        # Find all table rows in the parsed HTML
        tbl_quality_rows = soup.find_all('tr')

        # Initialize data list
        data_cost = []
        for row in tbl_quality_rows:
                row_data = []
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    row_data.append(cell.get_text().strip())
                data_cost.append(row_data)

        # Extract header and data rows separately
        header = ['kpi_name','kpi_value','kpi_hml']

        # Create DataFrame
        df_capital = pd.DataFrame(data_cost, columns=header)

        # Clean data
        df_capital['Index'] = idx
        df_capital['City'] = city_names[idx]
        df_capital.drop(columns=['kpi_hml'], inplace=True)
        df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
        # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
        df_capital.drop(8, inplace=True)


        # Transpose df
        df_capital_t = df_capital.pivot_table(index=['Index','City'], columns='kpi_name', values='kpi_value', dropna=False)
        df_capital_t.reset_index(inplace=True)
        df_capital_t.columns = df_capital_all.columns

        # Append data to complete DF
        df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

    except:
         print(city)

C:\Users\paude\AppData\Local\Temp\ipykernel_5684\3266304027.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)


Kabul-Afghanistan
Kandahar
Kandahar-Afghanistan
Durrës
Durrës-Albania
Tirana-Albania
Algiers-Algeria
Oran
Andorra-La-Vella-Andorra
Escaldes-Engordany
Escaldes-Engordany-Andorra
Huambo
Huambo-Angola
Luanda-Angola
All-Saints
All-Saints-Antigua-And-Barbuda
St-John%27S
St-John%27S-Antigua-And-Barbuda
Buenos-Aires-Argentina
Cordoba-Argentina
Gyumri
Gyumri-Armenia
Yerevan-Armenia
Adelaide-Australia
Brisbane-Australia
Gold-Coast-Australia
Melbourne-Australia
Perth-Australia
Sydney-Australia
Graz-Austria
Vienna-Austria
Baku-Azerbaijan
Ganja
Ganja-Azerbaijan
Manama-Bahrain
Riffa
Chittagong-Bangladesh
Dhaka-Bangladesh
Bridgetown-Barbados
Speightstown
Speightstown-Barbados
Gomel-Belarus
Minsk-Belarus
Antwerp-Belgium
Brussels-Belgium
Belize-City-Belize
Belmopan-Belize
Cotonou
Porto-Novo
Porto-Novo-Benin
Phuentsholing
Phuentsholing-Bhutan
Thimphu
La-Paz-Bolivia
Banja-Luka-Bosnia-And-Herzegovina
Sarajevo-Bosnia-And-Herzegovina
Francistown
Francistown-Botswana
Gaborone-Botswana
Brasilia-Brazil
São-Pa

In [10]:
# df_capital_all.head(100)
df_capital_all.shape

(794, 11)

In [14]:
df_capital_all.to_csv('test_out_final.csv')

In [214]:
# # Define the file path
# file_path = r'C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_cost.csv'

# # Export DataFrame to a CSV file
# df_capital.to_csv(file_path, index=False)

# print("DataFrame exported successfully to:", file_path)